In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.2 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import datasets

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [6]:
import pandas
data = pandas.read_csv('./All-seasons.csv')

In [7]:
data.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


In [8]:
from collections import Counter
from typing import List
import string

import seaborn
seaborn.set(palette = 'summer')

Тут я убираю переносы строк, знаки припинания, заглавные буквы. В общем, все, что будет нам мешать и не будет нести никакого смысла

In [9]:
d = []
for i in data['Line']:
    m = i.lower().translate(str.maketrans('', '', string.punctuation))
    d.append(m[:-1])
data['Line'] = d

In [10]:
data

,Season,Episode,Character,Line
0,10,1,Stan,you guys you guys chef is going away
1,10,1,Kyle,going away for how long
2,10,1,Stan,forever
3,10,1,Chef,im sorry boys
4,10,1,Stan,chef said hes been bored so he joining a group...
...,...,...,...,...
70891,9,14,Stan,i think youre pushing it
70892,9,14,Randy,how about twenty
70893,9,14,Stan,thats not disciprine
70894,9,14,Randy,right right does vodka count


In [26]:
data.shape

(70896, 4)

In [27]:
data.Character.value_counts()

Cartman          9774
Stan             7680
Kyle             7099
Butters          2602
Randy            2467
                 ... 
Reenactor 1         1
Confederate 2       1
Confederate 1       1
Army Doctor         1
Blind Man           1
Name: Character, Length: 3950, dtype: int64

In [28]:
data[data['Character']=='Cartman'].head(3)

,Season,Episode,Character,Line
9,10,1,Cartman,im gonna miss him im gonna miss chef and iand...
20,10,1,Cartman,reverse to you jew
25,10,1,Cartman,all right


Делю тут наши данные на учебные и тестовые и бью их тут по файлам, чтобы было проще с ними работать

In [29]:
train, test = train_test_split(data, test_size = 0.2)

In [30]:
with open('train.txt', 'w') as f:
    for each_text, each_label in zip(train['Line'], train['Character']):
        f.writelines(f'__label__{each_label} {each_text}\n')

with open('test.txt', 'w') as f:
    for each_text, each_label in zip(test['Line'], test['Character']):
        f.writelines(f'__label__{each_label} {each_text}\n')

In [31]:
!head -n 10 train.txt

__label__Chef well whats wrong with him
__label__Stan what would jesus do  yeheah 
__label__Shelly thanks for all your help eric you know as far as turds go youre okay 
__label__Cartman it didnt smell that bad you guys are overreacting
__label__Mr. Mackey the what
__label__French Delegate well we could give canada some small consolation prize
__label__Teen girl thanks a lot stan now were gonna be able to keep the youth center 
__label__Cartman i dont trust any of you assholes  where are you sons of bitches 
__label__Stuart kenny 
__label__Mrs. Garrison huy dont you dare kill my penis  crap weve gotta go back around


In [32]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199770 sha256=e308d25c1c75d9d0972c3fcaeb4c855692a8c1b7efad55a58183ada2d66cff55
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


Дальше я работаю с библиотекой fasttext, беру там разные модельки и проверяю, у каких будеть большая точность, третья оказалась самой рабочей и дальше я работаю с ней

In [33]:
import fasttext
model1 = fasttext.train_supervised('train.txt')

def print_results(sample_size, precision, recall):
    precision   = round(precision, 2)
    recall      = round(recall, 2)
    print(f'{sample_size=}')
    print(f'{precision=}')
    print(f'{recall=}')

print_results(*model1.test('test.txt'))



sample_size=14077
precision=0.25
recall=0.25


In [34]:
model4 = fasttext.train_supervised('train.txt', epoch=10, lr=1.0, wordNgrams =2)

print_results(*model4.test('test.txt'))

sample_size=14077
precision=0.32
recall=0.32


In [36]:
model5 = fasttext.train_supervised('train.txt', autotuneValidationFile='test.txt')

print_results(*model5.test('test.txt'))

sample_size=14077
precision=0.25
recall=0.25


In [37]:
model3 = fasttext.train_supervised('train.txt', epoch=10, lr=1.0)

print_results(*model3.test('test.txt'))

sample_size=14077
precision=0.33
recall=0.33


In [39]:
model3.save_model('optimized.model')

In [43]:
model = fasttext.load_model("optimized.model")

(('__label__Butters',), array([0.15610719]))

Тут я просто развлекаюсь и чекаю какие фразы будут какому персонажу приписываться

In [44]:
model.predict("I love studying at the university")

(('__label__Butters',), array([0.15610719]))

In [47]:
model.predict("It seems to me that I cheated a lot with the project")

(('__label__Craig',), array([0.13992603]))

In [46]:
model.predict("Oh no, they killed Kenny!")

(('__label__Stan',), array([0.54923147]))

In [48]:
model.predict("I'm terribly tired")

(('__label__Sharon',), array([0.80884629]))

In [49]:
model.predict("I don't understand anything about the documents that need to be submitted")

(('__label__Stan',), array([0.17381833]))